In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import warnings
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

# Utility stocks
TICKERS = ["AEP", "DUK", "SO", "ED", "EXC"]

# ตัวแปรภายนอก (SARIMAX)
EXOG_TICKER = "CL=F"


In [2]:
def best_arima(series, p_range=(0,2), d_range=(0,1), q_range=(0,2)):
    best_aic = np.inf
    best_order, best_model = None, None
    for p in range(p_range[0], p_range[1]+1):
        for d in range(d_range[0], d_range[1]+1):
            for q in range(q_range[0], q_range[1]+1):
                try:
                    model = ARIMA(series, order=(p,d,q)).fit()
                    if model.aic < best_aic:
                        best_aic, best_order, best_model = model.aic, (p,d,q), model
                except:
                    continue
    return best_order, best_aic, best_model


def best_sarima(series, 
                p_range=(0,1), d_range=(0,1), q_range=(0,1),
                P_range=(0,1), D_range=(0,1), Q_range=(0,1), m=12):
    best_aic = np.inf
    best_order, best_seasonal, best_model = None, None, None
    for p in range(p_range[0], p_range[1]+1):
        for d in range(d_range[0], d_range[1]+1):
            for q in range(q_range[0], q_range[1]+1):
                for P in range(P_range[0], P_range[1]+1):
                    for D in range(D_range[0], D_range[1]+1):
                        for Q in range(Q_range[0], Q_range[1]+1):
                            try:
                                model = SARIMAX(series,
                                                order=(p,d,q),
                                                seasonal_order=(P,D,Q,m),
                                                enforce_stationarity=False,
                                                enforce_invertibility=False).fit(disp=False)
                                if model.aic < best_aic:
                                    best_aic = model.aic
                                    best_order, best_seasonal, best_model = (p,d,q), (P,D,Q,m), model
                            except:
                                continue
    return best_order, best_seasonal, best_aic, best_model


def best_sarimax(series, exog, 
                 p_range=(0,1), d_range=(0,1), q_range=(0,1),
                 P_range=(0,1), D_range=(0,1), Q_range=(0,1), m=12):
    best_aic = np.inf
    best_order, best_seasonal, best_model = None, None, None
    for p in range(p_range[0], p_range[1]+1):
        for d in range(d_range[0], d_range[1]+1):
            for q in range(q_range[0], q_range[1]+1):
                for P in range(P_range[0], P_range[1]+1):
                    for D in range(D_range[0], D_range[1]+1):
                        for Q in range(Q_range[0], Q_range[1]+1):
                            try:
                                model = SARIMAX(series,
                                                exog=exog,
                                                order=(p,d,q),
                                                seasonal_order=(P,D,Q,m),
                                                enforce_stationarity=False,
                                                enforce_invertibility=False).fit(disp=False)
                                if model.aic < best_aic:
                                    best_aic = model.aic
                                    best_order, best_seasonal, best_model = (p,d,q), (P,D,Q,m), model
                            except:
                                continue
    return best_order, best_seasonal, best_aic, best_model


In [3]:
results = []

# โหลด exogenous
exog_df = yf.download(EXOG_TICKER, period="3y", auto_adjust=True, progress=False)[["Close"]].rename(columns={"Close":"EXOG"})

for ticker in TICKERS:
    print(f"\n=== {ticker} ===")
    df = yf.download(ticker, period="3y", auto_adjust=True, progress=False)
    series = df["Close"].dropna()
    
    # align exogenous
    exog = exog_df.loc[series.index].fillna(method="ffill")
    
    # --- ARIMA ---
    order_a, aic_a, _ = best_arima(series)
    
    # --- SARIMA ---
    order_s, seas_s, aic_s, _ = best_sarima(series)
    
    # --- SARIMAX ---
    order_x, seas_x, aic_x, _ = best_sarimax(series, exog)
    
    results.append({
        "Ticker": ticker,
        "ARIMA_order": order_a, "ARIMA_AIC": aic_a,
        "SARIMA_order": order_s, "SARIMA_seasonal": seas_s, "SARIMA_AIC": aic_s,
        "SARIMAX_order": order_x, "SARIMAX_seasonal": seas_x, "SARIMAX_AIC": aic_x,
    })

df_results = pd.DataFrame(results).set_index("Ticker")
df_results



=== AEP ===

=== DUK ===

=== SO ===

=== ED ===

=== EXC ===


,ARIMA_order,ARIMA_AIC,SARIMA_order,SARIMA_seasonal,SARIMA_AIC,SARIMAX_order,SARIMAX_seasonal,SARIMAX_AIC
Ticker,,,,,,,,
AEP,"(0, 1, 0)",2308.004019,"(0, 1, 1)","(1, 0, 1, 12)",2232.341772,"(0, 1, 1)","(1, 0, 1, 12)",2233.875064
DUK,"(0, 1, 0)",2305.200042,"(1, 1, 1)","(0, 1, 1, 12)",2220.034593,"(1, 1, 1)","(0, 1, 1, 12)",2214.699307
SO,"(0, 1, 2)",1919.884158,"(1, 1, 1)","(0, 1, 1, 12)",1858.965707,"(1, 1, 1)","(0, 1, 1, 12)",1856.651811
ED,"(0, 1, 0)",2252.826984,"(0, 1, 1)","(1, 0, 1, 12)",2193.814600,"(0, 1, 1)","(1, 0, 1, 12)",2186.014610
EXC,"(0, 1, 0)",1070.925120,"(1, 1, 1)","(1, 0, 1, 12)",1019.342349,"(1, 1, 1)","(1, 0, 1, 12)",1018.769048


In [4]:
def global_best(df, col_order, col_aic):
    # --- วิธี A: รวม AIC ---
    aic_sum = {}
    for order in df[col_order].unique():
        total = df.loc[df[col_order]==order, col_aic].sum()
        aic_sum[order] = total
    best_global = min(aic_sum, key=aic_sum.get)
    
    # --- วิธี B: Mode ---
    mode_order = Counter(df[col_order]).most_common(1)[0][0]
    
    return best_global, mode_order

# ARIMA
arima_global, arima_mode = global_best(df_results, "ARIMA_order", "ARIMA_AIC")
# SARIMA
sarima_global, sarima_mode = global_best(df_results, "SARIMA_order", "SARIMA_AIC")
# SARIMAX
sarimax_global, sarimax_mode = global_best(df_results, "SARIMAX_order", "SARIMAX_AIC")

print("Global Best Parameters:")
print(f"ARIMA  -> Best(sum AIC)={arima_global}, Mode={arima_mode}")
print(f"SARIMA -> Best(sum AIC)={sarima_global}, Mode={sarima_mode}")
print(f"SARIMAX-> Best(sum AIC)={sarimax_global}, Mode={sarimax_mode}")


Global Best Parameters:
ARIMA  -> Best(sum AIC)=(0, 1, 2), Mode=(0, 1, 0)
SARIMA -> Best(sum AIC)=(0, 1, 1), Mode=(1, 1, 1)
SARIMAX-> Best(sum AIC)=(0, 1, 1), Mode=(1, 1, 1)
